# Lispy

Veja o tutorial do Peter Norvig em http://norvig.com/lispy.html. Vamos implementar
um interpretador de Lisp/Scheme em Python.

In [14]:
# Imports

import math
import operator as op
from collections import deque, ChainMap


# Apelidos
Symbol = str    # símbolo na linguagem
Ast = list      # árvore sintática
Ctx = ChainMap  # contexto de execução

# Análise sintática + léxica

A principal função é eval_scheme(), que analisa uma string de código e retorna a árvore sintática correspondente.

In [10]:
def eval_scheme(st: str, ctx=default_context):
    """
    Avalia a string de código Scheme no contexto padrão.
    """
    
    ast = parse(st)
    return run_ast(ast, ChainMap({}, ctx))


def parse(st: str) -> Ast:
    """
    Realiza análise sintática da string de código e retorna uma 
    árvore sintática
    """
    tokens = lex(st)
    return parse_tokens(tokens)


def lex(st: str) -> deque:
    """
    Realiza a análise léxica e retorna uma lista de tokens a partir da
    string de código.
    """
    return deque(st.replace('(', ' ( ').replace(')', ' ) ').split())


def parse_tokens(tokens: deque) -> Ast:
    """
    Cria árvore sintática a partir de lista de tokens.
    """
    
    if tokens[0] == '(':
        del tokens[0]
        result = []
        while tokens[0] != ')':
            result.append(parse_tokens(tokens))
        del tokens[0]
        return result
    
    try:
        result = float(tokens[0])
        del tokens[0]
        return result
    except ValueError:
        return tokens.popleft()
    
    
# Contexto de execução
default_context = {
    '+': op.add,
    '-': op.sub,
    '*': op.mul,
    '/': op.truediv,
    '<': op.lt,
    '<=': op.le,
    'sqrt': math.sqrt,
}

# Interpretador

O interpretador consiste em uma única função que recebe uma árvore sintática e executa os comandos correspondentes no contexto dado.

In [15]:
def run_ast(ast: Ast, ctx: Ctx):
    """
    Executa árvore sintática no contexto de execução dada.
    """
    
    if isinstance(ast, (float, int)):
        return ast
    elif isinstance(ast, Symbol):
        try:
            return ctx[ast]
        except KeyError:
            raise NameError(f'unknown variable: {ast}')
    
    head, *args = ast
    if head == 'define':
        name, value = args
        ctx[name] = result = run_ast(value, ctx)
        return result

    elif head == 'if':
        cond, true, false = args
        cond = run_ast(cond, ctx)
        if cond:
            return run_ast(true, ctx)
        else:
            return run_ast(false, ctx)
        
    elif head == 'lambda':
        arg_names, body = args
        
        def function(*arg_values):
            local = dict(zip(arg_names, arg_values))
            new_ctx = ChainMap(local, ctx)
            return run_ast(body, new_ctx)
        return function
    
    else:
        func = run_ast(head, ctx)
        args = [run_ast(arg, ctx) for arg in args]
        return func(*args)

In [16]:
#
# Exemplos
#
scheme_factorial = '''
(define fat (lambda (n) 
    (if (< n 2) 
        1
        (* n (fat (- n 1))))))
'''

fat = eval_scheme(scheme_factorial)
fat(5)

120.0